# Imports

In [1]:
import numpy as np, pandas as pd
import os, shutil
from distutils.dir_util import copy_tree

In [2]:
f_path = './ml_vol/data/train/ratings_train.csv'

data = pd.read_csv(f_path)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3929542 entries, 0 to 3929541
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  int64  
 1   item_id  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 89.9 MB


## Make Volume to Mount with appropriate folders

In [3]:
mounted_volume = 'ml_vol'
if os.path.exists(mounted_volume): shutil.rmtree(mounted_volume)

os.mkdir(mounted_volume)
subdirs = ['data', 'logs', 'model', 'output', 'score']
for subdir in subdirs: 
    sub_dir_path = os.path.join(mounted_volume, subdir)
    os.mkdir(sub_dir_path)

## Copy Train and Test Data into Volume

## Define source path for train and test data

In [ ]:
'''
src_path should be a directory which has two folders: 1) train, and 2) test
'train' folder must have a file called: ratings_train.csv
'test' folder must have a file called : ratings_test.csv 
'''
dataset = 'jester'      # jester, movielens-10m, movielens-20m, book-crossing

src_path = f'./data/{dataset}/processed/'

## Copy Data from Source Path Into Mounted Volume

In [4]:
dest_path = os.path.join(mounted_volume, 'data')
for input_type in ['train', 'test']:
    full_src = os.path.join(src_path, input_type)
    full_dest = os.path.join(dest_path, input_type)
    
    if os.path.exists(full_src): copy_tree(full_src, full_dest)

# Build Image

In [10]:
%%bash 
docker build -t rec_base_mf:1 ./mf_gd

#1 [internal] load build definition from Dockerfile
#1 sha256:a8b89f33a23654abeb012fb71b01f9b6995a1cb207ca081f72a5b5f61c86c43c
#1 transferring dockerfile: 38B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:74f5ba0479ec426aa4c2c528eea1fe3b4852b175f40d77477ad9c919cde4af71
#2 transferring context: 34B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.8.0-slim
#3 sha256:4f0c597550e30aa54f707f0017cf64d137017976c13b147baa6fd4ad0c55c91e
#3 DONE 0.7s

#4 [1/5] FROM docker.io/library/python:3.8.0-slim@sha256:8e243f41e500238f78f7a29a81656114d3fe603d5c34079a462d090f71c4b225
#4 sha256:f2202870b184ece5b9a09c9b777f938cf0be25287ffe019e2c50e60191382ede
#4 DONE 0.0s

#5 [internal] load build context
#5 sha256:6437281d251671e62509e0ff3909bb72d850cdb3b4b658201f707bdb599ee234
#5 transferring context: 1.35kB 0.1s done
#5 DONE 0.1s

#6 [2/5] COPY ./requirements.txt .
#6 sha256:24d0411740d34b79ef29555fac8e75dfbc31cfca5f0dc2011cb48f1f5d0c6db7
#6 CACHED

#7 [3/5]

# Create Container From Image

In [13]:
%%bash 
declare vol_path="$(pwd)/ml_vol"
docker run -d -p 3000:3000 -v $vol_path:/app/ml_vol --name mfc rec_base_mf:1

b2926d12609c7e133edf169b79c5281232d727703d05a9f27dba76b7033c33d2


# Check Container and Image

In [14]:
%%bash 
docker ps

CONTAINER ID   IMAGE           COMMAND                  CREATED         STATUS         PORTS                    NAMES
b2926d12609c   rec_base_mf:1   "/bin/sh -c 'python â€¦"   9 seconds ago   Up 8 seconds   0.0.0.0:3000->3000/tcp   mfc


In [37]:
#%%bash 
#docker inspect mfc

# Run Training

In [17]:
%%bash 
docker exec mfc python train.py

Starting the training process...
train_data shape: (3929542, 3)
Preprocessing train_data ...
processed train_data shape: (3929542, 6)
Found # Users N = 73421; # Items M = 100 in training data
Doing train and validation split ...
Instantiating matrix factorizer ...
Fitting matrix factorizer ...
Epoch 1/30
27630/27630 [==============================] - 49s 2ms/step - loss: 0.0576 - mae: 0.1974 - val_loss: 0.0525 - val_mae: 0.1875
Epoch 2/30
27630/27630 [==============================] - 49s 2ms/step - loss: 0.0501 - mae: 0.1818 - val_loss: 0.0497 - val_mae: 0.1799
Epoch 3/30
27630/27630 [==============================] - 49s 2ms/step - loss: 0.0482 - mae: 0.1765 - val_loss: 0.0489 - val_mae: 0.1773
Epoch 4/30
27630/27630 [==============================] - 49s 2ms/step - loss: 0.0473 - mae: 0.1739 - val_loss: 0.0483 - val_mae: 0.1752
Epoch 5/30
27630/27630 [==============================] - 49s 2ms/step - loss: 0.0463 - mae: 0.1712 - val_loss: 0.0474 - val_mae: 0.1728
Epoch 6/30
27630/276

2021-12-04 23:51:32.248373: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-04 23:51:32.248423: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-04 23:51:35.372960: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-04 23:51:35.373000: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-04 23:51:35.373013: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b2926d12609c): /proc/driver/nvidia/version does not exist
2021-12-04 23:51:35.373126: I tensorflow/core/platform/cpu_featu

In [18]:
%%bash 
docker exec mfc python predict.py

test data shape:  (206818, 3)
proc_test_data shape:  (206818, 6)
preds shape:  (206818, 1)


2021-12-05 00:20:38.882138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-05 00:20:38.882176: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-12-05 00:20:39.811835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-05 00:20:39.811875: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-05 00:20:39.811887: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b2926d12609c): /proc/driver/nvidia/version does not exist
2021-12-05 00:20:39.812037: I tensorflow/core/platform/cpu_featu